In [1]:
TMVA::Tools::Instance();
TString outfileName( "TMVAReg.root" );
TFile* outputFile = TFile::Open( outfileName, "RECREATE" );

TMVA::Factory *factory = new TMVA::Factory( "TMVARegression", outputFile,
                                               "!V:!Silent:Color:DrawProgressBar:AnalysisType=Regression" );

TMVA::DataLoader *dataloader=new TMVA::DataLoader("dataset");

for(UInt_t i=0;i<162;i++)
dataloader->AddVariable( Form("img[%d]",i));
 


In [2]:
for(UInt_t i=0;i<162;i++)
dataloader->AddVariable( Form("img[%d]",i));
 
dataloader->AddTarget( "target" );

In [3]:
auto input = TFile::Open("regtrain.root");

TTree *reg = (TTree*)input->Get("reg");

// global event weights per tree (see below for setting event-wise weights)
Double_t regWeight  = 1.0;

dataloader->AddRegressionTree( reg, regWeight );
 

DataSetInfo              : [dataset] : Added class "Regression"
                         : Add Tree reg of type Regression with 36000 events


In [ ]:
dataloader->PrepareTrainingAndTestTree( "",
                                         "nTrain_Regression=36000:nTest_Regression=0:SplitMode=Random:NormMode=NumEvents:!V" );
  

                         : Dataset[dataset] : Class index : 0  name : Regression


In [ ]:
TString layoutString("Layout=TANH|100,LINEAR");

TString training0("LearningRate=1e-5,Momentum=0.5,Repetitions=1,ConvergenceSteps=500,BatchSize=50,"
                        "TestRepetitions=7,WeightDecay=0.01,Regularization=NONE,DropConfig=0.5+0.5+0.5+0.5,"
                        "DropRepetitions=2, Multithreading=True");

TString trainingStrategyString("TrainingStrategy=");
trainingStrategyString += training0 ;

//       TString trainingStrategyString
//       ("TrainingStrategy=LearningRate=1e-1,Momentum=0.3,Repetitions=3,ConvergenceSteps=20,BatchSize=30,TestRepetitions=7,WeightDecay=0.0,L1=false,DropFraction=0.0,DropRepetitions=5");

TString nnOptions("!H:V:ErrorStrategy=SUMOFSQUARES:VarTransform=G:WeightInitialization=XAVIERUNIFORM:Architecture=CPU");
nnOptions.Append(":");
nnOptions.Append(layoutString);
nnOptions.Append(":");
nnOptions.Append(trainingStrategyString);

factory->BookMethod(dataloader, TMVA::Types::kDNN, "DNN_CPU", nnOptions); // NN

In [ ]:
factory->BookMethod( dataloader,  TMVA::Types::kBDT, "BDTG",
                           "!H:!V:NTrees=2000::BoostType=Grad:Shrinkage=0.1:UseBaggedBoost:BaggedSampleFraction=0.5:nCuts=20:MaxDepth=3:MaxDepth=4" );
